In [18]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv("segmented_1.csv")
df = df.drop(columns=['company', 'year','pena_n'])
df = df.dropna(axis = 0,how = 'any')

def data_append(data,n):
    newdf = pd.DataFrame(np.repeat(data.values,n,axis=0))
    newdf.columns = df.columns
    return newdf

In [19]:
df

,text_a,label
0,前一年度營業結果營業實施成果本公司於104年10月15日與中國信託金融控股股份有限公司(以下...,Y
1,"051億元。上年度預算執行情形:合併元年因進行整併,公司營運得以發揮綜效,預算執行情形良好,...",Y
2,"﻿前一年度營業結果105年度營業計畫實施成果105年,受新興市場成長動能放緩、美日歐表現不如...",Y
3,"420億元,年成長率為3%。預算執行情形：不適用。財務收支及獲利能力分析本公司105年營業收...",Y
4,"﻿105年度營業結果回顧105年,受惠於國際電子產品需求增溫、原油及原物料價格止跌回穩,出口...",Y
...,...,...
207,達成預算目標之156%。◎以合併財務報告為編製基礎。財務收支及獲利能力分析106年度利息淨收...,Y
208,"106年度營業報告經濟局勢回顧106年,幾乎所有經濟調研機構在經濟報告上用字遣詞都漸趨樂觀,...",N
209,"對報酬不宜過度樂觀。加上美國、歐元區、日本、中國等主要國家債務累積速度持續快於資本,在金融海...",N
210,"106年度營業結果國內外金融環境106年,我國出口動能持續增溫,帶動經濟表現優於預期,全年經...",Y


In [20]:
df_train = data_append(df.head(10),1)
df_test = data_append(df.tail(3),1)

df_train.to_csv("train.tsv", sep="\t", index=False)
print("訓練樣本數：", len(df_train))

df_test.to_csv("test.tsv", sep="\t", index=False)
print("預測樣本數：", len(df_test))

訓練樣本數： 10
預測樣本數： 3


In [21]:
from torch.utils.data import Dataset
import torch
from transformers import BertTokenizer,AutoTokenizer
from IPython.display import clear_output

class FakeNewsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要驗證集
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'Y': 0, 'N': 1}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a = self.df.iloc[idx, :1].values
            label_tensor = None
        else:
            text_a, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a
        len_a = len(word_pieces)
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        return (tokens_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

# tokenizer = AutoTokenizer.from_pretrained("schen/longformer-chinese-base-4096")
trainset = FakeNewsDataset("train", tokenizer=tokenizer)

In [22]:
for i in iter(trainset):
    data = i
    tokens_tensors,label_tensor = data
    print(tokens_tensors.shape)

torch.Size([318])
torch.Size([333])
torch.Size([288])
torch.Size([399])
torch.Size([851])
torch.Size([1729])
torch.Size([204])
torch.Size([316])
torch.Size([1271])
torch.Size([1471])


In [23]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    
    # 測試集有 labels
    if samples[0][1] is not None:
        label_ids = torch.stack([s[1] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    # attention masks，將 tokens_tensors 裡頭不為 zero padding的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, masks_tensors, label_ids



BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [24]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text_a, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text_a}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

label_tensor   ：{label_tensor}

--------------------

""")

[原始文本]
句子 1：前一年度營業結果營業實施成果本公司於104年10月15日與中國信託金融控股股份有限公司(以下簡稱中信金控)進行股份轉換,並成為中信金控持股百分之百之子公司。中信金控自100年11月接手大都會國際人壽(該公司於101年1月更名為中國信託人壽保險股份有限公司,以下簡稱中信人壽),中信人壽又於103年併購宏利人壽台灣分公司。為有效整合資源,以提升綜效,105年1月1日中信金控旗下兩家壽險子公司中信人壽與台灣人壽合併,資產已破兆,105年整體營運狀況良好,茲將經營績效報告如下:初年度保費為1,399億元,業界排名第4名,銀行保險、保險經紀人及代理人、直效行銷通路皆排名業界前三名,且業務部隊更加擴大,擁有超過九千名業務員。總保費為2,524億,業界排名第5名。可運用資金12
分類  ：Y

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 1184,  671, 2399, 2428, 4245, 3511, 5178, 3362, 4245, 3511, 2179,
        3177, 2768, 3362, 3315, 1062, 1385, 3176, 8503, 2399, 8108, 3299, 8115,
        3189, 5645,  704, 1751,  928, 6249, 7032, 6084, 2971, 5500, 5500,  819,
        3300, 7361, 1062, 1385,  113,  809,  678, 5080, 4935,  704,  928, 7032,
        2971,  114, 6868, 6121, 5500,  819, 6752, 2994,  117,  699, 2768, 4158,
         704,  928, 7032, 2971, 2898, 5500, 4636, 1146,  722, 4636,  722, 2094,
        1062, 1385,  511,  704,  928, 7032, 2971, 5632, 8135, 2399, 8111, 3299,
        2970, 2797, 192

In [25]:
data = next(iter(trainloader))

tokens_tensors,masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([10, 1729]) 
tensor([[ 101, 1184,  671,  ...,    0,    0,    0],
        [ 101, 8137, 8148,  ...,    0,    0,    0],
        [ 101, 1184,  671,  ...,    0,    0,    0],
        ...,
        [ 101, 3315, 3309,  ...,    0,    0,    0],
        [ 101, 1184,  671,  ...,    0,    0,    0],
        [ 101, 2769,  947,  ...,    0,    0,    0]])
------------------------
masks_tensors.shape    = torch.Size([10, 1729])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        = torch.Size([10])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])



In [26]:
for i in iter(trainloader):
    data = i
    tokens_tensors,masks_tensors, label_ids = data
    
    print(tokens_tensors.shape)
    print(masks_tensors.shape)
    print(label_ids.shape)
    
#     print('tokens_tensors: '+ tokens_tensors.shape)
#     print('masks_tensors: '+ masks_tensors.shape)
#     print('label_ids: '+ label_ids.shape)

torch.Size([10, 1729])
torch.Size([10, 1729])
torch.Size([10])


In [27]:
from transformers import BertForSequenceClassification,AutoModel

PRETRAINED_MODEL_NAME = "schen/longformer-chinese-base-4096"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [28]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, masks_tensors = data[:2]
            outputs = model(input_ids=tokens_tensors, attention_mask=masks_tensors)    
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[2]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cpu
classification acc: 0.8


In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [30]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：105021698
線性分類器的參數量：1538



In [31]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 3  
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors,masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1434931680 bytes.